# Setting Up the Environment
This section installs and imports the required libraries and packages for running the Delft3D-FLOW and Delft3D-FM simulations.
If you are running this notebook for the first time, ensure the necessary packages are installed.

In [ ]:
# Install required packages (run this once if packages are missing)
!pip install numpy matplotlib xarray datetime tqdm pandas

In [ ]:
# Install additional tools for Delft3D (requires internet connection)
!pip install git+https://github.com/openearth/dfm_tools.git

# Importing Libraries
This section imports all the necessary libraries to perform data handling, visualization, and Delft3D-specific operations.

In [ ]:
import sys
import numpy as np
import pandas as pd
import openearthtools.io.delft3d as d3d
import xarray as xr
import datetime
from tqdm.notebook import tnrange
import warnings

# Suppress warnings for a cleaner output
warnings.filterwarnings("ignore")

# Reading and Processing Delft3D-FLOW Files
This section reads the required input files for Delft3D-FLOW simulations, such as grid, depth, layer, and boundary files.
It calculates boundary positions and sets up variables for further processing.

In [ ]:
# Specify the directory containing Delft3D-FLOW input files
path = 'Delft3D-FLOW Folder/'

# Load the grid, depth, and boundary information
G = d3d.Grid.fromfile(path + '*.grd')
D = pd.DataFrame(d3d.Dep.read(path + '*.dep', G.shape).val)
lyr = pd.read_csv(path + '*.lyr', sep='\s+', header=None).T
lyr.columns = ['depth']

# Read boundary definitions
df = pd.read_csv(path + '*.bnd', sep='\s+', header=None)
df.columns = ['name_boundary', 'type_boundary', 'type_data', 'A_x', 'A_y', 'B_x', 'B_y', 'alpha', 'name_A', 'name_B']

# Initialize lists for boundary coordinates
x_A, y_A, x_B, y_B = [], [], [], []

# Calculate boundary coordinates
for k in range(len(df)):
    if 'East' in df['name_boundary'].loc[k]:
        ind_y_A, ind_x_A = df['A_y'].loc[k], df['A_x'].loc[k] - 2
        ind_y_B, ind_x_B = df['B_y'].loc[k], df['B_x'].loc[k] - 2
    elif 'North' in df['name_boundary'].loc[k]:
        ind_y_A, ind_x_A = df['A_y'].loc[k] - 2, df['A_x'].loc[k]
        ind_y_B, ind_x_B = df['B_y'].loc[k] - 2, df['B_x'].loc[k]
    else:
        ind_y_A, ind_x_A = df['A_y'].loc[k], df['A_x'].loc[k]
        ind_y_B, ind_x_B = df['B_y'].loc[k], df['B_x'].loc[k]

    x_A.append(G.__dict__['x'][ind_y_A][ind_x_A])
    y_A.append(G.__dict__['y'][ind_y_A][ind_x_A])
    x_B.append(G.__dict__['x'][ind_y_B][ind_x_B - 1])
    y_B.append(G.__dict__['y'][ind_y_B][ind_x_B - 1])

df['x_A'], df['y_A'], df['x_B'], df['y_B'] = x_A, y_A, x_B, y_B

# Processing NETCDF Files
This section loads the NETCDF files, selects the required time slice, and prepares the data for salinity and temperature calculations.

In [ ]:
# Load the NETCDF file
nc_fileID = input('Input netcdf file name: ')
ds = xr.open_dataset(nc_fileID)

# Filter dataset for the desired time range
ds = ds.sel({'time': slice('2019-01-01T00:00:00.000000000', '2021-01-01T00:00:00.000000000')})

# Output File Generation
This section creates output files for Delft3D boundary conditions using salinity and temperature data.

In [ ]:
# Specify output format
fmt = '.bcc'
outfile_name = 'name_of_file' + fmt
outfile = open(outfile_name, 'w')

# Process boundaries and write output
for j in tnrange(len(df), desc='Processing Boundaries'):
    depths = (lyr * D.iloc[df['A_y'].loc[0], df['A_x'].loc[0]] / 200).cumsum(axis=0)
    
    # Write metadata and parameters to file
    outfile.write('table-name           ' + r"'Boundary Section : " + str(j + 1) + r"'\n")
    outfile.write('contents             ' + r"'3d-profile'\n")
    ...

print('File generation completed.')